In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import utils

In [2]:
train_iter,test_iter = utils.load_data(256,4)

In [3]:
class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs, num_hiddens):
        super(Net, self).__init__()
        self.l1 = nn.Linear(num_inputs, num_hiddens)
        self.relu1 = nn.ReLU()
        self.l2 = nn.Linear(num_hiddens, num_outputs)
        
    def forward(self, X):
        X = X.view(X.shape[0], -1)
        o1 = self.relu1(self.l1(X))
        o2 = self.l2(o1)
        return o2
    
    def init_params(self):
        for param in self.parameters():
            init.normal_(param, mean=0, std=0.01)

In [4]:
net = Net(28*28, 10, 256)
net.init_params()

In [5]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=0.5)

In [6]:
for epoch in range(5):
    train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y).sum()
        l.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        train_l_sum += l.item()
        train_acc_sum += (y_hat.argmax(dim=1) == y).sum().item()
        n += y.shape[0]
    test_acc = utils.evaluate_accuracy(test_iter, net)
    print('epoch %d, loss %.4f, train_acc %.3f，test_acc %.3f' % (epoch + 1, train_l_sum / n, train_acc_sum / n, test_acc))

epoch 1, loss 0.0032, train_acc 0.699，test_acc 0.765
epoch 2, loss 0.0019, train_acc 0.821，test_acc 0.816
epoch 3, loss 0.0017, train_acc 0.842，test_acc 0.787
epoch 4, loss 0.0015, train_acc 0.855，test_acc 0.801
epoch 5, loss 0.0015, train_acc 0.863，test_acc 0.852
